In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
from scipy.special import gamma, gammainc
from scipy import stats

In [9]:
Y = np.arange (10)
X = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
n = 100

# а) Подсчет Колмогорова

In [10]:
X_sample = []
for i in range (len(X)):
    for j in range (X[i]):
        X_sample.append (i)

In [15]:
F_acc = stats.uniform.cdf (Y, loc=-0.5, scale=10)

F_emp = []
for i in Y:
    F_emp.append (np.sum(X_sample < i) / n)
    
delta = (n**0.5) * max ( [ abs(F_acc[i] - F_emp[i]) for i in range (len(Y)) ] )
print (delta)

1.5999999999999996


# б) Подсчет Хи2 и Колмогорова

In [16]:
limits = [-np.inf]
for i in range (len(Y) - 1):
    limits.append (i + 0.5)
limits.append (np.inf)

print (limits)

[-inf, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, inf]


In [18]:
def normal_pdf (x, a, sigma):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp( -(x - a)**2 / (2 * sigma**2))

# Нужно прикинуть, где могут находиться параметры a и sigma
a = np.linspace (1, 10, 100)
sigma = np.linspace (1, 10, 100)

In [25]:
L_max = 0
a_norm = 0
sigma_norm = 0

for sigma_i in sigma:
    for a_i in a:
        L_i = 1
        
        for i in range (len(X)):
            p_i = integrate.quad (normal_pdf, limits[i], limits[i + 1], args = (a_i, sigma_i))[0]
            L_i *= p_i ** X[i]
        
        if L_i > L_max:
            L_max = L_i
            sigma_norm = sigma_i
            a_norm = a_i

In [30]:
print ("Найденные параметры распределения: a -", a_norm, "sigma -", sigma_norm)

Найденные параметры распределения: a - 4.818181818181818 sigma - 2.6363636363636367


In [34]:
delta = 0
for i in range (len(X)):
    pi = integrate.quad (normal_pdf, limits[i], limits[i + 1], args = (a_norm, sigma_norm))[0]
    delta += (X[i] - n * pi)**2 / (n * pi)
print ("Вычисленная дельта -", delta)

Вычисленная дельта - 9.837954716903855


In [36]:
N = 50000
delta_K = []

for i in range (N):
    sample_i = np.random.normal (a_norm, sigma_norm, n)
    tmp = np.linspace (0, max(max(sample_i), 9), 100)
    
    F_emp_star = [np.sum(sample_i < j) / n for j in tmp]
    F_emp = [np.sum(X_sample < j) / n for j in tmp]
    
    delta_i = (n ** 0.5) * max ([abs(F_emp_star[i] - F_emp[i]) for i in range (len(tmp))])
    delta_K.append (delta_i)

In [38]:
delta_wave = (n ** 0.5) * np.max (abs([np.sum(X_sample < j) / n for j in Y] - stats.norm (a_norm, sigma_norm).cdf(Y)))
print (delta_wave)

0.8626400707993842


In [39]:
delta_K_var = np.sort (delta_K)

k = len (delta_K_var [delta_K_var < delta_wave])
p_value = 1 - ((k - 1)/N)
print (p_value)

0.99316


# с) Хи2 метод моментов

In [41]:
a_mm = np.mean (X_sample)
sigma_mm = np.std (X_sample)

print (a_mm, sigma_mm)

4.77 2.505414137423193


In [45]:
delta_mm = []
N = 10000

for i in range (N):
    sample_i = np.random.normal (a_mm, sigma_mm, n)
    mean_i = np.mean (sample_i)
    sigma_i = np.std (sample_i)
    
    pi = []
    mi = []
    for j in range (len (X)):
        pi.append (integrate.quad (normal_pdf, limits[j], limits[j + 1], args = (mean_i, sigma_i))[0])
        mi.append (len (sample_i [(sample_i >= limits[j]) & (sample_i <= limits[j + 1])]))
    delta_mm.append(sum ( ((mi[i] - n * pi[i]) ** 2) / (n * (pi[i])) for i in range (len(X))))

In [48]:
delta_mm_var = np.sort (delta_mm)
p_value = 1 - ((len(delta_mm_var[delta_mm_var < delta]) - 1) / N)
print (p_value, "Нет основания отвергнуть гипотезу")

0.20930000000000004 Нет основания отвергнуть гипотезу
